In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Project import Project

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary

import pandas as pd

from Scripts.data.DataLoading import get_image_dataloaders, get_image_dataset
from Scripts.data.SSVEPDataset import SSVEPDataset
from Scripts.data.SSVEPDataloader import SSVEPDataloader
from Scripts.neuralnets.NNPreinstalledModelSelection import *
from Scripts.neuralnets.NNTrainingUtils import train, kfold_train

In [4]:
ngpu = 1; device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [5]:
from Scripts.data.BenchmarkDataset import BenchmarkDataset
from Scripts.data.Dataloader import Dataloader

In [25]:
dataset = BenchmarkDataset(Project, classes = list(range(1, 20 + 1)))

In [26]:
batch_size = 256
signal_length = 500
dataloaders, dataset_sizes = Dataloader(dataset, batch_size)
print(dataset_sizes)

{'train': 9792, 'val': 2448}


***

In [8]:
model_type = 'resnet'; model_size = '18'; num_classes = 5; model_name = model_type + model_size
model = model_selection(model_name, num_classes)
model.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
model = model.to(device)

In [9]:
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [ ]:
model, stats = kfold_train(Project, model, dataset, criterion, optimizer, num_epochs = 50, num_folds = 5, batch_size = batch_size)

***

In [8]:
from Scripts.neuralnets.NNTrainingUtils import train_autoencoder

In [6]:
from Models.Autoencoder1 import Autoencoder

In [27]:
model = Autoencoder(kernel_n = 2, stride = , n_feature_maps = 1).to(device)
summary(model, (1, 64, int(signal_length)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 1, 64, 500]               6
              ReLU-2           [-1, 1, 64, 500]               0
         MaxPool2d-3  [[-1, 1, 64, 250], [-1, 1, 64, 250]]               0
       BatchNorm2d-4           [-1, 1, 64, 250]               2
       MaxUnpool2d-5           [-1, 1, 64, 500]               0
   ConvTranspose2d-6           [-1, 1, 64, 500]               6
Total params: 14
Trainable params: 14
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.12
Forward/backward pass size (MB): 1952.03
Params size (MB): 0.00
Estimated Total Size (MB): 1952.15
----------------------------------------------------------------


In [28]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)
# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

In [29]:
model, stats = train_autoencoder(Project, model, dataloaders, dataset_sizes, criterion, optimizer, num_epochs = 100)

***

In [13]:
class HybridModel(nn.Module):
    def __init__(self, classifier, encoder):
        super(HybridModel, self).__init__()
        self.encoder = encoder
        self.classifier = classifier
        
    def forward(self, x):
        with torch.set_grad_enabled(False):
            x = self.encoder(x)
        
        x = self.classifier(x)
        
        return x

In [30]:
model_type = 'resnet'; model_size = '18'; num_classes = 20; model_name = model_type + model_size
classifier = model_selection(model_name, num_classes)
classifier.conv1 = nn.Conv2d(1, 64, kernel_size = (7, 7), stride = (2, 2), padding = (3, 3), bias = False)
encoder = model.encoder
hybrid = HybridModel(classifier, encoder).to(device)

In [31]:
optimizer = optim.SGD(hybrid.parameters(), lr = 0.001, momentum = 0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)
criterion = nn.CrossEntropyLoss()

In [32]:
hybrid, stats = kfold_train(Project, hybrid, dataset, criterion, optimizer, num_epochs = 50, num_folds = 5)